<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection-/blob/main/Human_Activity_Detection_mobilenet_v3_large_075_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'
test_data = pd.read_csv(test_csv_path)

In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CSV files
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Training_set.csv'
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/testing_set.csv'

train_data = pd.read_csv(train_csv_path)
test_data = pd.read_csv(test_csv_path)

# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 32

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
    rotation_range=20,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift the width by 20%
    height_shift_range=0.2,  # Randomly shift the height by 20%
    horizontal_flip=True  # Randomly flip images horizontally
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/train',  # Directory containing the train images
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Check the class indices assigned by the generator
class_indices = test_generator.class_indices
print(class_indices)



Found 12100 validated image filenames belonging to 15 classes.
Found 500 validated image filenames belonging to 15 classes.
{'calling': 0, 'clapping': 1, 'cycling': 2, 'dancing': 3, 'drinking': 4, 'eating': 5, 'fighting': 6, 'hugging': 7, 'laughing': 8, 'listening_to_music': 9, 'running': 10, 'sitting': 11, 'sleeping': 12, 'texting': 13, 'using_laptop': 14}


In [5]:
import tensorflow as tf
import tensorflow_hub as hub

# Define the model
m = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])


# Build the model with the default input shape
m.build((None, 224, 224, 3))  # Batch input shape.

# Print the model summary
print(m.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2731616   
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dense_1 (Dense)             (None, 15)                3855      
                                                                 
Total params: 3,063,407
Trainable params: 3,044,319
Non-trainable params: 19,088
_________________________________________________________________
None


In [6]:

m.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
m.fit(train_generator, epochs=10,batch_size=batch_size)

Epoch 1/10
379/379 [==============================] - 2682s 7s/step - loss: 1.9355 - accuracy: 0.5877
Epoch 2/10
379/379 [==============================] - 1804s 5s/step - loss: 1.5113 - accuracy: 0.7193
Epoch 3/10
379/379 [==============================] - 1780s 5s/step - loss: 1.3373 - accuracy: 0.7722
Epoch 4/10
379/379 [==============================] - 1781s 5s/step - loss: 1.2358 - accuracy: 0.7998
Epoch 5/10
379/379 [==============================] - 1772s 5s/step - loss: 1.1377 - accuracy: 0.8233
Epoch 6/10
379/379 [==============================] - 1773s 5s/step - loss: 1.0719 - accuracy: 0.8421
Epoch 7/10
379/379 [==============================] - 1783s 5s/step - loss: 1.0357 - accuracy: 0.8518
Epoch 8/10
379/379 [==============================] - 1776s 5s/step - loss: 0.9901 - accuracy: 0.8635
Epoch 9/10
379/379 [==============================] - 1781s 5s/step - loss: 0.9425 - accuracy: 0.8764
Epoch 10/10
379/379 [==============================] - 1791s 5s/step - loss: 0.908

In [7]:




# Evaluate the model on the test generator
test_loss, test_accuracy = m.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

16/16 [==============================] - 101s 6s/step - loss: 1.8526 - accuracy: 0.7080
Test Loss: 1.8525725603103638
Test Accuracy: 0.7080000042915344


In [8]:

m.save_weights('mobiletNetv3large.h5')

In [9]:

from google.colab import files
files.download('mobiletNetv3large.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>